# Simulation & Parameter Estimation of SEIR Model

SEIR is a type of compartmental models which are used in modelling of infectious disease using differential equations. These types of models divide the population into groups or compartments and the dynamics of these groups are expressed with the help of a system of differential equations.

These system of equations are parametrized which capture the mechanistic nature of the disease. For simulation, you select values of these parameters and the resulting curves simulate the behaviour by solving the set of equations. Finally the results are plotted in a graph to visually understand the effect of the parameters.

## SEIR Model

For completeness the SEIR model is produced below:

<img src="images/seir_model.png">

$\displaystyle \frac{dS}{dt} = -\frac{\beta S I}{N}$<br><br>
$\displaystyle \frac{dE}{dt} = \frac{\beta S I}{N} - \sigma E$<br><br>
$\displaystyle \frac{dI}{dt} = \sigma E -  \gamma I$<br><br>
$\displaystyle \frac{dR}{dt} = \gamma I$<br><br>
$N = S + E + I + R$<br><br>
Where,<br><br>
$\beta$ is infection rate or the rate of spread<br><br>
$\sigma$ is the incubation rate or the rate of latent individuals becoming infectious (average duration of incubation is $1/\sigma$)<br><br>
$\gamma$ is the recovery rate or mortality rate. If the duration of indection is D then $\gamma$ = 1/D

#### A supplementary blogpost to this notebook can be found at [Estimating Parameters of Compartmental Models from Observed Data](https://dilbertai.com/2020/04/27/estimating-parameters-of-compartmental-models-from-observed-data/)

In [17]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
import requests
from lmfit import minimize, Parameters, Parameter, report_fit
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')

In [18]:
# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout, ToggleButton

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [19]:
def ode_model(z, t, beta, sigma, gamma):
    """
    Reference https://www.idmod.org/docs/hiv/model-seir.html
    """
    S, E, I, R = z
    N = S + E + I + R
    dSdt = -beta*S*I/N
    dEdt = beta*S*I/N - sigma*E
    dIdt = sigma*E - gamma*I
    dRdt = gamma*I
    return [dSdt, dEdt, dIdt, dRdt]

In [20]:
def ode_solver(t, initial_conditions, params):
    initE, initI, initR, initN = initial_conditions
    beta, sigma, gamma = params['beta'].value, params['sigma'].value, params['gamma'].value
    initS = initN - (initE + initI + initR)
    res = odeint(ode_model, [initS, initE, initI, initR], t, args=(beta, sigma, gamma))
    return res

In [21]:
response = requests.get('https://api.rootnet.in/covid19-in/stats/history')
print('Request Success? {}'.format(response.status_code == 200))
covid_history = response.json()['data']

Request Success? True


In [22]:
keys = ['day', 'total', 'confirmedCasesIndian', 'confirmedCasesForeign', 'confirmedButLocationUnidentified',
        'discharged', 'deaths']
df_covid_history = pd.DataFrame([[d.get('day'), 
                                  d['summary'].get('total'), 
                                  d['summary'].get('confirmedCasesIndian'), 
                                  d['summary'].get('confirmedCasesForeign'),
                                  d['summary'].get('confirmedButLocationUnidentified'),
                                  d['summary'].get('discharged'), 
                                  d['summary'].get('deaths')] 
                                 for d in covid_history],
                    columns=keys)
df_covid_history = df_covid_history.sort_values(by='day')
df_covid_history['infected'] = df_covid_history['total'] - df_covid_history['discharged'] - df_covid_history['deaths']
df_covid_history['total_recovered_or_dead'] = df_covid_history['discharged'] + df_covid_history['deaths']

In [23]:
# ref: https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf
initN = 1380000000
# S0 = 966000000
initE = 1000
initI = 47
initR = 0
sigma = 1/5.2
gamma = 1/2.9
R0 = 4
beta = R0 * gamma
days = 112

params = Parameters()
params.add('beta', value=beta, min=0, max=10)
params.add('sigma', value=sigma, min=0, max=10)
params.add('gamma', value=gamma, min=0, max=10)

## Simulation

In [24]:
def main(initE, initI, initR, initN, beta, sigma, gamma, days, param_fitting):
    initial_conditions = [initE, initI, initR, initN]
    params['beta'].value, params['sigma'].value,params['gamma'].value = [beta, sigma, gamma]
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, E, I, R = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3]
    
    # Create traces
    fig = go.Figure()
    if not param_fitting:
        fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers', name='Susceptible'))
        fig.add_trace(go.Scatter(x=tspan, y=E, mode='lines+markers', name='Exposed'))
    fig.add_trace(go.Scatter(x=tspan, y=I, mode='lines+markers', name='Infected'))
    fig.add_trace(go.Scatter(x=tspan, y=R, mode='lines+markers',name='Recovered'))
    if param_fitting:
        fig.add_trace(go.Scatter(x=tspan, y=df_covid_history.infected, mode='lines+markers',\
                             name='Infections Observed', line = dict(dash='dash')))
        fig.add_trace(go.Scatter(x=tspan, y=df_covid_history.total_recovered_or_dead, mode='lines+markers',\
                             name='Recovered/Deceased Observed', line = dict(dash='dash')))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    # Edit the layout
    fig.update_layout(title='Simulation of SEIR Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    if not os.path.exists("images"):
        os.mkdir("images")
    fig.write_image("images/seir_simulation.png")
    fig.show()

In [25]:
interact(main, initE=IntSlider(min=0, max=100000, step=1, value=initE, description='initE', style=style, \
                               layout=slider_layout),
               initI=IntSlider(min=0, max=100000, step=10, value=initI, description='initI', style=style, \
                               layout=slider_layout),
               initR=IntSlider(min=0, max=100000, step=10, value=initR, description='initR', style=style, \
                               layout=slider_layout),
               initN=IntSlider(min=0, max=1380000000, step=1000, value=initN, description='initN', style=style, \
                               layout=slider_layout),
               beta=FloatSlider(min=0, max=4, step=0.01, value=beta, description='Infection rate', style=style, \
                                layout=slider_layout),
               sigma=FloatSlider(min=0, max=4, step=0.01, value=sigma, description='Incubation rate', style=style, \
                                 layout=slider_layout),
               gamma=FloatSlider(min=0, max=4, step=0.01, value=gamma, description='Recovery rate', style=style, \
                                 layout=slider_layout),
               days=IntSlider(min=0, max=600, step=7, value=days, description='Days', style=style, \
                              layout=slider_layout),
               param_fitting=ToggleButton(value=False, description='Fitting Mode', disabled=False, button_style='', \
             tooltip='Click to show fewer plots', icon='check-circle')
        );

interactive(children=(IntSlider(value=1000, description='initE', layout=Layout(width='99%'), max=100000, style…

## Parameter Estimation

In [26]:
def error(params, initial_conditions, tspan, data):
    sol = ode_solver(tspan, initial_conditions, params)
    return (sol[:, 2:4] - data).ravel()

In [55]:
initial_conditions = [initE, initI, initR, initN]
beta = 1.08
sigma = 0.02
gamma = 0.02
params['beta'].value = beta
params['sigma'].value = sigma
params['gamma'].value = gamma
days = 100
tspan = np.arange(0, days, 1)
data = df_covid_history.loc[0:(days-1), ['infected', 'total_recovered_or_dead']].values

In [56]:
params

name,value,initial value,min,max,vary
beta,1.08000000,1.3793103448275863,0.00000000,10.0000000,True
sigma,0.02000000,0.1923076923076923,0.00000000,10.0000000,True
gamma,0.02000000,0.3448275862068966,0.00000000,10.0000000,True


In [57]:
# fit model and find predicted values
result = minimize(error, params, args=(initial_conditions, tspan, data), method='leastsq')

In [58]:
result.params

name,value,standard error,relative error,initial value,min,max,vary
beta,0.11448092,0.00112784,(0.99%),1.08,0.00000000,10.0000000,True
sigma,1.79360114,0.02096219,(1.17%),0.02,0.00000000,10.0000000,True
gamma,0.05753477,0.00105321,(1.83%),0.02,0.00000000,10.0000000,True


In [59]:
# display fitted statistics
report_fit(result)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 251
    # data points      = 200
    # variables        = 3
    chi-square         = 1.5205e+10
    reduced chi-square = 77180382.3
    Akaike info crit   = 3635.30845
    Bayesian info crit = 3645.20340
[[Variables]]
    beta:   0.11448092 +/- 0.00112784 (0.99%) (init = 1.08)
    sigma:  1.79360114 +/- 0.02096219 (1.17%) (init = 0.02)
    gamma:  0.05753477 +/- 0.00105321 (1.83%) (init = 0.02)
[[Correlations]] (unreported correlations are < 0.100)
    C(beta, gamma)  =  0.937
    C(beta, sigma)  = -0.521
    C(sigma, gamma) = -0.212


In [61]:
final = data + result.residual.reshape(data.shape)
fig = go.Figure()
fig.add_trace(go.Scatter(x=tspan, y=data[:, 0], mode='markers', name='Observed Infections', line = dict(dash='dot')))
fig.add_trace(go.Scatter(x=tspan, y=data[:, 1], mode='markers', name='Observed Recovered/Deceased', line = dict(dash='dot')))
fig.add_trace(go.Scatter(x=tspan, y=final[:, 0], mode='lines+markers', name='Fitted Infections'))
fig.add_trace(go.Scatter(x=tspan, y=final[:, 1], mode='lines+markers', name='Fitted Recovered/Deceased'))
fig.update_layout(title='SEIR: Observed vs Fitted',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=1000, height=600
                     )

In [39]:
observed_IR = df_covid_history.loc[:, ['infected', 'total_recovered_or_dead']].values
print(observed_IR.shape)

(458, 2)


In [48]:
val_days = 450
tspan_fit_pred = np.arange(0, val_days, 1)
params['beta'].value = result.params['beta'].value
params['sigma'].value = result.params['sigma'].value
params['gamma'].value = result.params['gamma'].value
fitted_predicted = ode_solver(tspan_fit_pred, initial_conditions, params)

In [49]:
fitted_predicted_IR = fitted_predicted[:, 2:4]
print(fitted_predicted_IR.shape)

(450, 2)


In [54]:
# val_days = 200
# tspan = np.arange(0, val_days, 1)
# observed_IRD = df_covid_history.loc[:, ['infected', 'discharged', 'deaths']].values
data = df_covid_history.loc[0:(val_days-1), ['infected', 'discharged', 'deaths']].values

fig = go.Figure()
fig.add_trace(go.Scatter(x=fitted_predicted_IR, y=data[:, 0], mode='markers', name='Observed Infections', line = dict(dash='dot')))
# fig.add_trace(go.Scatter(x=tspan, y=data[:, 1], mode='markers', name='Observed Recovered', line = dict(dash='dot')))
# fig.add_trace(go.Scatter(x=tspan, y=data[:, 2], mode='markers', name='Observed Deaths', line = dict(dash='dot')))
fig.add_trace(go.Scatter(x=fitted_predicted_IR, y=fitted_predicted_IR[:, 0], mode='lines+markers', name='Fitted Infections'))
# fig.add_trace(go.Scatter(x=tspan, y=fitted_predicted_IRD[:, 1], mode='lines+markers', name='Fitted Recovered'))
# fig.add_trace(go.Scatter(x=tspan, y=fitted_predicted_IRD[:, 2], mode='lines+markers', name='Fitted Deaths'))
fig.update_layout(title='VSEIRD: Observed vs Fitted（{} Days)'.format(val_days),
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=1000, height=600
                     )

In [52]:
print("Fitted MAE")
print('Infected: ', np.mean(np.abs(fitted_predicted_IR[:days, 0] - observed_IR[:days, 0])))
print('Recovered/Deceased: ', np.mean(np.abs(fitted_predicted_IR[:days, 1] - observed_IR[:days, 1])))

print("\nFitted RMSE")
print('Infected: ', np.sqrt(np.mean((fitted_predicted_IR[:days, 0] - observed_IR[:days, 0])**2)))
print('Recovered/Deceased: ', np.sqrt(np.mean((fitted_predicted_IR[:days, 1] - observed_IR[:days, 1])**2)))

Fitted MAE
Infected:  836.9923389687799
Recovered/Deceased:  387.2454115091947

Fitted RMSE
Infected:  983.7427962189978
Recovered/Deceased:  467.7038942354901


In [44]:
print("Predicted MAE")
print('Infected: ', np.mean(np.abs(fitted_predicted_IR[days:observed_IR.shape[0], 0] - observed_IR[days:, 0])))
print('Recovered/Deceased: ', np.mean(np.abs(fitted_predicted_IR[days:observed_IR.shape[0], 1] - observed_IR[days:, 1])))

print("\nPredicted RMSE")
print('Infected: ', np.sqrt(np.mean((fitted_predicted_IR[days:observed_IR.shape[0], 0] - observed_IR[days:, 0])**2)))
print('Recovered/Deceased: ', np.sqrt(np.mean((fitted_predicted_IR[days:observed_IR.shape[0], 1] - observed_IR[days:, 1])**2)))

Predicted MAE
Infected:  139249357.809918
Recovered/Deceased:  788231711.7877078

Predicted RMSE
Infected:  241865100.87343892
Recovered/Deceased:  980949032.1110216


In [45]:
interact(main, initE=IntSlider(min=0, max=100000, step=1, value=initE, description='initE', style=style, \
                               layout=slider_layout),
               initI=IntSlider(min=0, max=100000, step=10, value=initI, description='initI', style=style, \
                               layout=slider_layout),
               initR=IntSlider(min=0, max=100000, step=10, value=initR, description='initR', style=style, \
                               layout=slider_layout),
               initN=IntSlider(min=0, max=1380000000, step=1000, value=initN, description='initN', style=style, \
                               layout=slider_layout),
               beta=FloatSlider(min=0, max=4, step=0.01, value=result.params['beta'].value, description='Infection rate', style=style, \
                                layout=slider_layout),
               sigma=FloatSlider(min=0, max=4, step=0.01, value=result.params['sigma'].value, description='Incubation rate', style=style, \
                                 layout=slider_layout),
               gamma=FloatSlider(min=0, max=4, step=0.01, value=result.params['gamma'].value, description='Recovery rate', style=style, \
                                 layout=slider_layout),
               days=IntSlider(min=0, max=600, step=7, value=240, description='Days', style=style, \
                              layout=slider_layout),
               param_fitting=ToggleButton(value=False, description='Fitting Mode', disabled=True, button_style='', \
             tooltip='Click to show fewer plots', icon='check-circle')
        );

interactive(children=(IntSlider(value=1000, description='initE', layout=Layout(width='99%'), max=100000, style…

**References:**<br>
1. SEIR and SEIRS Model https://www.idmod.org/docs/hiv/model-seir.html<br>
2. Compartmental models in epidemiology https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SEIR_model<br>
3. Solve Differential Equations in Python https://www.youtube.com/watch?v=VV3BnroVjZo<br>
4. Computational Statistics in Python https://people.duke.edu/~ccc14/sta-663/CalibratingODEs.html<br>
5. Ordinary Differential Equations (ODE) with Python and Jupyter https://elc.github.io/posts/ordinary-differential-equations-with-python/<br>
6. SEIRS+ Model https://github.com/ryansmcgee/seirsplus<br>
7. Stack Overflow https://stackoverflow.com/questions/40753159/why-is-scipy-minimize-ignoring-my-constraints<br>
8. Lotka–Volterra equations https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations<br>
9. SEIR and Regression Model based COVID-19 outbreak predictions in India https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf<br>

A simulator built with RShiny which provides many more parameters https://alhill.shinyapps.io/COVID19seir/